# Asthma Deaths - Processing

## Dependencies

Ensure that the required libraries have been installed locally as per the README.md file included in this project.

Run the following cell the import the required dependencies for this notebook.

In [1]:
# Import libraries
import pandas as pd

## Load Dataset

In [2]:
# Load the raw data.
df = pd.read_csv('raw.csv')

## Exploratory Analysis of Raw Data

In [3]:
# Generate summary statistics for object columns.
object_summary_stats = df.describe(include=['O']).transpose()                                                       # Generate summary statistics for object columns.
object_summary_stats['missing_values'] = df.isnull().sum()                                                          # Add missing values to the summary table.
object_summary_stats['present_values'] = df.notnull().sum()                                                         # Add present values to the summary table.
object_summary_stats['datatype'] = df.dtypes                                                                        # Add data types to the summary table.
object_summary_stats = object_summary_stats[['datatype', 'present_values', 'missing_values', 'unique']]             # Select features and reorder table.

# Display the summary tables with titles.
print("Dataset Head:")                                                                                              # Display the dataset head title.
display(df.head().style.set_table_styles([{'selector': 'th', 'props': [('min-width', '100px')]}]))                  # Display the dataset head. For better visualization, set the minimum width of the table headers to 100px.

print("\nObject Summary Statistics:")                                                                               # Display the object summary statistics.
display(object_summary_stats.style.set_table_styles([{'selector': 'th', 'props': [('min-width', '100px')]}]))       # Display the object summary statistics. For better visualization, set the minimum width of the table headers to 100px.

Dataset Head:


,Asthma Type,LHD,Period,Per cent,LL 95% CI,UL 95% CI
0,Current Asthma,Sydney LHD,2002-2004,11.700000,7.600000,15.800000
1,Current Asthma,Sydney LHD,2003-2005,10.700000,6.600000,14.800000
2,Current Asthma,Sydney LHD,2004-2006,13.200000,8.600000,17.800000
3,Current Asthma,Sydney LHD,2005-2007,12.500000,8.000000,17.000000
4,Current Asthma,Sydney LHD,2006-2008,14.600000,9.900000,19.300000



Object Summary Statistics:


,datatype,present_values,missing_values,unique
Asthma Type,object,512,0,2
LHD,object,512,0,16
Period,object,512,0,16


## Data Manipulation

In [4]:
# Rename columns to match Air Quality data set.
df = df.rename(columns={'LHD': 'lhd', 'Period': 'date'})                                                            # Rename columns
df.columns = df.columns.str.lower()                                                                                 # Lowercase column names

# Remove ' LHD' for Local Health District values.
df['lhd'] = df['lhd'].str.replace(' LHD', '')                                                                       # Remove ' LHD' from lhd column

# Remove rows representing state-wide aggregated data.
df = df.dropna(subset=['lhd'])                                                                                      # Remove rows with missing values in 'lhd' column
df = df[~df['lhd'].str.contains('All')]                                                                             # Remove rows with 'All' in 'lhd' column

# Remove columns holding Confidence Interval data.
df = df.loc[:, ~df.columns.str.contains('% ci')]                                                                    # Remove columns with '% ci' in the name

# Convert 'per cent' to 'rate per 100,000 population'.
df = df.rename(columns={'per cent': 'rate per 100,000 population'})                                                 # Rename 'per cent' column
df['rate per 100,000 population'] = df['rate per 100,000 population'] * 1000                                        # Remove ' per cent' from 'rate per 100,000 population' column

# View the cleaned data.
display(df.head())                                                                                                  # Display the first few rows of the cleaned data

,asthma type,lhd,date,"rate per 100,000 population"
0,Current Asthma,Sydney,2002-2004,11700.0
1,Current Asthma,Sydney,2003-2005,10700.0
2,Current Asthma,Sydney,2004-2006,13200.0
3,Current Asthma,Sydney,2005-2007,12500.0
4,Current Asthma,Sydney,2006-2008,14600.0


## Data Normalization

Convert rolling 2-Year totals to annual totals.

In [5]:
yearly_data = []                                                                                                   # Create a list to hold the new data.

# Ensure column names are stripped of leading/trailing spaces
df.columns = df.columns.str.strip()

# Iterate over the rows.
for index, row in df.iterrows():                                                                                   # Iterate over the rows.
    start_year, end_year = map(int, row['date'].split('-'))                                                        # Get the start and end years.
    mid_year = start_year + 1                                                                                      # Calculate the mid year.
    split_value = row['rate per 100,000 population'] / 2                                                           # Calculate the split value.

    yearly_data.extend([
        {'lhd': row['lhd'], 'financial year': f"{start_year}/{mid_year}", 'rate per 100,000 population': split_value},         # Append the first financial year of the period.
        {'lhd': row['lhd'], 'financial year': f"{mid_year}/{end_year}", 'rate per 100,000 population': split_value}            # Append the second financial year of the period.
    ])
    
# Create a new DataFrame.
df_yearly = pd.DataFrame(yearly_data)                                                                              # Create a new DataFrame.

# Group by 'lhd' and 'financial year'. Get an average of the 'value' column.
df_yearly = df_yearly.groupby(['financial year', 'lhd']).mean().reset_index()                                      # Group by 'lhd' and 'financial year'. Get an average of the 'value' column.

# Assign to original DataFrame.
df = df_yearly                                                                                                     # Assign to original DataFrame.

# View the cleaned data.
display(df.head())                                                                                                 # Display the first few rows of the cleaned data

,financial year,lhd,"rate per 100,000 population"
0,2002/2003,Central Coast,12025.0
1,2002/2003,Far West,12450.0
2,2002/2003,Hunter New England,11725.0
3,2002/2003,Illawarra Shoalhaven,10200.0
4,2002/2003,Mid North Coast,10775.0


## Set Date Range

Set the range of financial years from 2014/2015 to 2023/2024.

In [6]:
# Drop pre 2014/2015 data.
df = df[~df['financial year'].isin(['2011/2012', '2012/2013', '2013/2014'])]                                        # Drop pre 2014/2015 data.

# Add rows for each LHD for the missing years until 2023/2024.
missing_rows = []                                                                                                   # Create a list to store the missing rows.
lhds = df['lhd'].unique()                                                                                           # Get unique LHDs.
years = [f"{year}/{year + 1}" for year in range(2014, 2024)]                                                        # Create a list of years from 2014/2015 to 2023/2024.

for lhd in lhds:
    for year in years:
        if not ((df['lhd'] == lhd) & (df['financial year'] == year)).any():                                         # Check if the row is missing.
            missing_rows.append({'lhd': lhd, 'financial year': year, 'rate per 100,000 population': None})          # Append the missing row to the list.

# Create a DataFrame from the missing rows and concatenate it to the original DataFrame
if missing_rows:
    df_missing = pd.DataFrame(missing_rows)                                                                         # Create a DataFrame from the missing rows.
    df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.

# Sort the DataFrame by 'lhd' and 'financial year'.
df = df.sort_values(by=['lhd', 'financial year']).reset_index(drop=True)                                            # Sort the DataFrame by 'lhd' and 'financial year'.

# View the DataFrame.
display(df.head())                                                                                                  # View the last 5 rows of the DataFrame.

C:\Users\matth\AppData\Local\Temp\ipykernel_2740\979375999.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_missing], ignore_index=True)                                                             # Concatenate the DataFrames.


,financial year,lhd,"rate per 100,000 population"
0,2002/2003,Central Coast,12025.0
1,2003/2004,Central Coast,12200.0
2,2004/2005,Central Coast,12900.0
3,2005/2006,Central Coast,12662.5
4,2006/2007,Central Coast,11737.5


Fill missing values using linear interpolation.

In [7]:
df['rate per 100,000 population'] = df['rate per 100,000 population'].interpolate()                                # Fill missing values using linear interpolation.

## Output Processed Dataset

In [8]:
# Save the processed data to a new CSV file.
df.to_csv('processed.csv', index=False)                                                                            # Save the processed data to a new CSV file.                                                                                                        # View the first 5 rows of the DataFrame.

## View Processed Dataset

In [9]:
# Display the processed data.
display(df.head())

,financial year,lhd,"rate per 100,000 population"
0,2002/2003,Central Coast,12025.0
1,2003/2004,Central Coast,12200.0
2,2004/2005,Central Coast,12900.0
3,2005/2006,Central Coast,12662.5
4,2006/2007,Central Coast,11737.5
